In [1]:
import tensorflow as tf

# Variable Scope


你可以如在 [Variables HowTo](https://www.tensorflow.org/versions/r0.10/how_tos/variable_scope/index.html)中所说的那样创建一个变量，并对其初始化，保存和加载。但是当你在构建一个非常复杂的模型的时候，你经常会需要共享很多的变量，并且你可能需要同时初始化所有的变量。这些都可以通过variable scope来实现。

# Variable sharing

假设我们要有一个函数能够对图像进行处理，我们的输入是两张图片，对这两张图片先分别调用图像处理函数，然后在对两张图片的处理结果进行合并。但是我们要注意到tensorflow的不同之处在于，其每一步操作都会被附加到graph上，调用一次函数，该函数内部的所有操作都会被附加到graph上，而多次调用这些操作就会被附加多次。


我们可以考虑以下这个图像处理函数：

```python
def my_image_filter(input_images):
    conv1_weights = tf.Variable(tf.random_normal([5, 5, 32, 32]),
        name="conv1_weights")
    conv1_biases = tf.Variable(tf.zeros([32]), name="conv1_biases")
    conv1 = tf.nn.conv2d(input_images, conv1_weights,
        strides=[1, 1, 1, 1], padding='SAME')
    relu1 = tf.nn.relu(conv1 + conv1_biases)

    conv2_weights = tf.Variable(tf.random_normal([5, 5, 32, 32]),
        name="conv2_weights")
    conv2_biases = tf.Variable(tf.zeros([32]), name="conv2_biases")
    conv2 = tf.nn.conv2d(relu1, conv2_weights,
        strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv2 + conv2_biases)
```


如果我们分别对image1和image2 调用一次图像处理函数，那么就会产生8个变量，
```python
# First call creates one set of 4 variables.
result1 = my_image_filter(image1)
# Another set of 4 variables is created in the second call.
result2 = my_image_filter(image2)
```

但是我们想让这两个图像被同样的参数处理，我们可以将变量部分和操作部分分开：

```python
variables_dict = {
    "conv1_weights": tf.Variable(tf.random_normal([5, 5, 32, 32]),
        name="conv1_weights")
    "conv1_biases": tf.Variable(tf.zeros([32]), name="conv1_biases")
    ... etc. ...
}

def my_image_filter(input_images, variables_dict):
    conv1 = tf.nn.conv2d(input_images, variables_dict["conv1_weights"],
        strides=[1, 1, 1, 1], padding='SAME')
    relu1 = tf.nn.relu(conv1 + variables_dict["conv1_biases"])

    conv2 = tf.nn.conv2d(relu1, variables_dict["conv2_weights"],
        strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv2 + variables_dict["conv2_biases"])

# The 2 calls to my_image_filter() now use the same variables
result1 = my_image_filter(image1, variables_dict)
result2 = my_image_filter(image2, variables_dict)
```


## Variable scope

为了解决上述的变量共享的问题，tensorflow提供了variable scope。

其主要是由两个函数组成：

### * tf.get_variable(name, shape, initializer): Creates or returns a variable with a given name.

### * tf.variable_scope(scope_name): Manages namespaces for names passed to tf.get_variable().



### 理解 tf.get_variable()

我们通常这样来调用
```python
v = tf.get_variable(name, shape, dtype, initializer)
```

取决于该函数被调用时所在的variable scope，有两种不同的可能：

1. 该scope被设定为创建新对象，此时 tf.get_variable_scope().reuse == False

    在这种情况下，v会被创建为一个新对象，其shape和dtype由参数所决定。新创建的对象的名称被设定为 scope的name + 函数的参数name。
    
    并且tensorflow会做一个检查，确保这个新对象的名称之前不存在，否则这个函数会发生ValueError。
    
    ```python
    with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1])
    assert v.name == "foo/v:0"
    ```
2. 该scope被设定为reuse 变量，此时 tf.get_variable_scope().reuse == True
    
    在这种情况下，tensorflow会去搜索scope name + 函数的参数name组成的变量name是否存在，如果存在则返回该变量，如果不存在则抛出ValueError异常
    
    ```python
    with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1])
    with tf.variable_scope("foo", reuse=True):
    v1 = tf.get_variable("v", [1])
    assert v1 == v
    ```
   
    
    

### 理解tf.variable_scope()


该函数可以接受一个name做为该scope的name，并且可以接受一个reuse-flag来决定是否reuse variable。同时，scope可以嵌套：

```python
with tf.variable_scope("foo"):
    with tf.variable_scope("bar"):
        v = tf.get_variable("v", [1])
assert v.name == "foo/bar/v:0"
```


#### 获取 当前scope & 将reuse flag 设置为True
我们可以通过tf.get_variable_scope()函数来获取当前的variable scope，并且可以通过调用tf.get_variable_scope().reuse_variables():

 函数来将该scope的reuse flag设置为True
 
```python
with tf.variable_scope("foo"):
    v = tf.get_variable("v", [1])
    tf.get_variable_scope().reuse_variables()
    v1 = tf.get_variable("v", [1])
assert v1 == v
```

### variable scope 的reuse设置成True之后不能设置为False

注意你不能讲reuse flag设置成False。这样限定的目的是为了我们在创建模型时候能够将函数组合起来。假设一个人在一个reuse被设置成True的scope里面调用一个函数，他肯定期望这个函数里面的所有变量都是reuse的。如果我们能够将reuse设置成False，就会破坏这种一致性。

虽然我们不可以显示地将reuse设置成显示地，但是我们可以进入一个reusing scope然后退出来。

> 如果一个scope是reuse的，那么其subscope都是reuse的

```python
with tf.variable_scope("root"):
    # At start, the scope is not reusing.
    assert tf.get_variable_scope().reuse == False
    with tf.variable_scope("foo"):
        # Opened a sub-scope, still not reusing.
        assert tf.get_variable_scope().reuse == False
    with tf.variable_scope("foo", reuse=True):
        # Explicitly opened a reusing scope.
        assert tf.get_variable_scope().reuse == True
        with tf.variable_scope("bar"):
            # Now sub-scope inherits the reuse flag.
            assert tf.get_variable_scope().reuse == True
    # Exited the reusing scope, back to a non-reusing one.
    assert tf.get_variable_scope().reuse == False
```




### variable scope 的捕获


在之前的例子中，我们都是用name这个参数来生成一个variable scope，但是如果我们能捕获这个scope就可以每次不通过name来获取这个scope


```python
with tf.variable_scope("foo") as foo_scope:
    v = tf.get_variable("v", [1])
with tf.variable_scope(foo_scope)
    w = tf.get_variable("w", [1])
with tf.variable_scope(foo_scope, reuse=True)
    v1 = tf.get_variable("v", [1])
    w1 = tf.get_variable("w", [1])
assert v1 == v
assert w1 == w
```

### 重新打开一个被捕获scope的时候就会跳出当前的scope

> 注意必须是被捕获的scope，并不是name参数获得的scope

```python
with tf.variable_scope("foo") as foo_scope:
    assert foo_scope.name == "foo"
with tf.variable_scope("bar")
    with tf.variable_scope("baz") as other_scope:
        assert other_scope.name == "bar/baz"
        with tf.variable_scope(foo_scope) as foo_scope2:
            assert foo_scope2.name == "foo"  # Not changed.
```

In [2]:
with tf.variable_scope("foo"):
    assert tf.get_variable_scope().name =="foo"
with tf.variable_scope("bar"):
    with tf.variable_scope("baz") as other_scope:
        assert other_scope.name == "bar/baz"
        with tf.variable_scope("foo") as foo_scope:
            print foo_scope.name
            assert foo_scope.name == "foo"

bar/baz/foo


AssertionError: 

In [ ]:
with tf.variable_scope("foo") as foo_scope:
    assert foo_scope.name == "foo"
with tf.variable_scope("bar"):
    with tf.variable_scope("baz") as other_scope:
        assert other_scope.name == "bar/baz"
        with tf.variable_scope(foo_scope) as foo_scope2:
            assert foo_scope2.name == "foo"  # Not changed.

### variable scope 的初始器(initializer)

我们可以为一个scope指定initializer，这个scope里面的subscope也会继承这个initializer。但是为某个操作更明确地指定另一个initializer会覆盖默认的initializer

```python
with tf.variable_scope("foo", initializer=tf.constant_initializer(0.4)):
    v = tf.get_variable("v", [1])
    assert v.eval() == 0.4  # Default initializer as set above.
    w = tf.get_variable("w", [1], initializer=tf.constant_initializer(0.3)):
    assert w.eval() == 0.3  # Specific initializer overrides the default.
    with tf.variable_scope("bar"):
        v = tf.get_variable("v", [1])
        assert v.eval() == 0.4  # Inherited default initializer.
    with tf.variable_scope("baz", initializer=tf.constant_initializer(0.2)):
        v = tf.get_variable("v", [1])
        assert v.eval() == 0.2  # Changed default initializer.
```

### variable scope对其中的operation的name的影响

我们每次调用 with tf.variable_scope("name")的时候，相当于默认执行了 tf.name_scope("name"). 

```python
with tf.variable_scope("foo"):
    x = 1.0 + tf.get_variable("v", [1])
assert x.op.name == "foo/add"
```


而且在一个variable scope 里面还可以open一个name scope

```python
with tf.variable_scope("foo"):
    with tf.name_scope("bar"):
        v = tf.get_variable("v", [1])
        x = 1.0 + v
assert v.name == "foo/v:0"
assert x.op.name == "foo/bar/add"
```

> 注意这里只有operation x 的name上加上了 name scope 而变量 v却没有,因为 v是由调用get_variable得到的, 这个函数会忽略name scope，只会对variable scope敏感。



In [ ]:

with tf.variable_scope("foo"):
    with tf.name_scope("bar"):
        y = tf.get_variable("y", [1])
        w = tf.Variable([[1.0]])
        x = 1.0 + y

print "get_variable will ignore name_scope"
assert y.name == "foo/y:0"


print w.name

assert x.op.name == "foo/bar/add"

### variable scope 和name scope

variable scope对于variable 以及operation都有用，而name scope只是对operation有用。另外，get_variable会无视name scope


```python
with tf.Graph().as_default() as g:
  c = tf.constant(5.0, name="c")
  assert c.op.name == "c"
  c_1 = tf.constant(6.0, name="c")
  assert c_1.op.name == "c_1"

  # Creates a scope called "nested"
  with g.name_scope("nested") as scope:
    nested_c = tf.constant(10.0, name="c")
    assert nested_c.op.name == "nested/c"

    # Creates a nested scope called "inner".
    with g.name_scope("inner"):
      nested_inner_c = tf.constant(20.0, name="c")
      assert nested_inner_c.op.name == "nested/inner/c"

    # Create a nested scope called "inner_1".
    with g.name_scope("inner"):
      nested_inner_1_c = tf.constant(30.0, name="c")
      assert nested_inner_1_c.op.name == "nested/inner_1/c"

      # Treats `scope` as an absolute name scope, and
      # switches to the "nested/" scope.
      with g.name_scope(scope):
        nested_d = tf.constant(40.0, name="d")
        assert nested_d.op.name == "nested/d"

        with g.name_scope(""):
          e = tf.constant(50.0, name="e")
          assert e.op.name == "e"
```